In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from treeinterpreter import treeinterpreter as ti

from sklearn.metrics import classification_report, roc_auc_score

/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [130]:
aa.keys()

dict_keys(['늪깔따구아과류', '네모집날도래 Kub', '민강도래 KUb', '둥근입기수우렁이', '큰애우묵날도래', '청나비날도래', '애모기 KUa', '금빛하루살이', '깔따구류 sp.1', '작은강하루살이', '각다귀 KUa', '두갈래하루살이', '애날도래 Kua', '명주각다귀 Kua', '긴무늬왕잠자리', '큰밀잠자리', '늪깔따구류', '넓은물삿갓벌레 KUa', '참거머리', '갈색우묵날도래 Kub', '뿔하루살이', '큰자실잠자리', '꼬마물방개', '수염치레날도래', '알물땡땡이', '고추좀잠자리', '밀잠자리붙이', '둥근날개날도래', '뒷가시물땡땡이', '몽땅하루살이', '아가미지렁이', '광택날도래 KUa', '중국물진드기', '참재첩', '실지렁이류', '동양줄날도래', '꼬리치레하루살이', '민하루살이', '깔따구류', '곳체다슬기', '별날도래', '보통옆새우', '애넓적물땡땡이', '개똥하루살이', '쇠측범잠자리', '흰부채하루살이', '검은머리물날도래', '애날도래 KUa', '연못하루살이', '실지렁이', '등검은실잠자리', '갈색우묵날도래 KUa', '물벌레류', '물벌레1', '각다귀류', '말조개', '네점하루살이', '소금쟁이', '얼룩날개모기류', '둥근물삿갓벌레 KUa', '물달팽이', '고추잠자리', '줄동애등에', '어리장수잠자리', '두점하루살이', '잔산잠자리', '아시아실잠자리', '새뱅이', '다슬기', '왕잠자리', '방울실잠자리'])

In [136]:
bird_xgb = pickle.load(open('../model_dict_weight_adj/bird_xgb_exclude.pickle','rb'))
flr_xgb = pickle.load(open('../model_dict_weight_adj/flr_xgb.pickle','rb'))
bnin_xgb = pickle.load(open('../model_dict_weight_adj/bnin_xgb_exclude.pickle','rb'))
insect_xgb = pickle.load(open('../model_dict_weight_adj/insect_xgb_exclude.pickle','rb'))

# bird_rf = pickle.load(open('../model_dict_weight_adj/bird_rf.pickle','rb'))
# flr_rf = pickle.load(open('../model_dict_weight_adj/flr_rf.pickle','rb'))
# bnin_rf = pickle.load(open('../model_dict_weight_adj/bnin_rf.pickle','rb'))
# insect_rf = pickle.load(open('../model_dict_weight_adj/insect_rf.pickle','rb'))

bird_data = pickle.load(open('../model_dict_weight_adj/bird_data.pickle','rb'))
flr_data = pickle.load(open('../model_dict_weight_adj/flr_data.pickle','rb'))
bnin_data = pickle.load(open('../model_dict_weight_adj/bnin_data_exclude.pickle','rb'))
insect_data = pickle.load(open('../model_dict_weight_adj/insect_data.pickle','rb'))

In [133]:
len(bird_xgb.keys()), len(flr_xgb.keys()), len(bnin_xgb.keys()), len(insect_xgb.keys()) 

(10, 80, 68, 59)

In [146]:
dict_model_list = [bird_xgb, flr_xgb, bnin_xgb, insect_xgb]
dict_data_list = [bird_data, flr_data, bnin_data, insect_data]
dict_roc_auc = {}
cat = ['bird','flr','bnin','insect']
for c in cat:
    dict_roc_auc[c] = []

for n, m, d in zip(cat, dict_model_list, dict_data_list):
    for key in list(m.keys()):
        model = m[key]
        X_test = d[key]['X_test']
        y_test = d[key]['y_test']
        pred = model.predict(X_test)
        score = roc_auc_score(y_test, pred)
        dict_roc_auc[n].append(score)

In [147]:
for i, key in enumerate(list(dict_roc_auc.keys())):
    print(f'{key}에서 최대값은 {max(dict_roc_auc[key])}, 평균은 {np.mean(dict_roc_auc[key])}')
    print(f'종은 { list(dict_model_list[i].keys())[np.where(dict_roc_auc[key] == max(dict_roc_auc[key]))[0][0]] }')

bird에서 최대값은 0.9733266590389017, 평균은 0.8744297449615643
종은 섬휘파람새
flr에서 최대값은 0.9673076923076923, 평균은 0.7819836339918538
종은 참식나무
bnin에서 최대값은 0.9938650306748466, 평균은 0.7734038892599281
종은 늪깔따구아과류
insect에서 최대값은 0.9191629955947136, 평균은 0.7723515877176464
종은 제주왕딱정벌레


## prediction time!

In [113]:
pred_grid = pd.read_csv('../concat_data/pred_grid/final_test_data.csv')

In [124]:
## incheon
inch = pd.read_csv('../concat_data/pred_grid/inch_final.csv')
inch = inch.fillna(0)
model = bnin_xgb['깔따구류']
y_pred = model.predict(inch)
y_prob = model.predict_proba(inch)
inch['y_pred'] = y_pred
inch['y_prob'] = y_prob[:,1]
inch.to_csv('../concat_data/pred_grid/inch_final_pred.csv', index=False)

In [137]:
## bnin
df_pred = pd.DataFrame()
df_prob = pd.DataFrame()

# for key in list(bnin_xgb.keys()):
#     model = bnin_xgb[key]
#     y_pred = model.predict(pred_grid.iloc[:,:-2])
#     y_prob = model.predict_proba(pred_grid.iloc[:,:-2])
    
#     df_pred[key] = y_pred
#     df_prob[key] = y_prob[:,1]
    
# df_pred['location'] = pred_grid['location']
# df_pred['grid_index'] = pred_grid['grid_index']

# df_prob['location'] = pred_grid['location']
# df_prob['grid_index'] = pred_grid['grid_index']

# df_pred = pd.DataFrame()
# df_prob = pd.DataFrame()

## bird
for key in list(insect_xgb.keys()):
    model = insect_xgb[key]
    y_pred = model.predict(pred_grid.iloc[:,:-2])
    y_prob = model.predict_proba(pred_grid.iloc[:,:-2])
    
    df_pred[key] = y_pred
    df_prob[key] = y_prob[:,1]
    
df_pred['location'] = pred_grid['location']
df_pred['grid_index'] = pred_grid['grid_index']

df_prob['location'] = pred_grid['location']
df_prob['grid_index'] = pred_grid['grid_index']

In [139]:
df_pred.to_csv('../concat_data/pred_grid/insect/df_pred.csv', index=False)
df_prob.to_csv('../concat_data/pred_grid/insect/df_prob.csv', index=False)

In [145]:
df_prob['알락하늘소'].min()

0.39338257908821106

## tree interpreter

In [113]:
X = train_test_specs_dict['섬휘파람새'][2].iloc[np.array([26,298]),:]
rf_model = bird_rf['섬휘파람새']
for i,row in X.iterrows():

    data_point = pd.DataFrame([row])
    data_point.set_axis(['value_variable']) # Once transposed, it will be the column name
    prediction, bias, contributions = ti.predict(rf_model, data_point)
    local_interpretation = data_point.append(
    pd.DataFrame([[round(c[1],3) for c in contributions[0]]], columns=data_point.columns.tolist(), index=['contribution_variable'])
    ).T.sort_values('contribution_variable', ascending=False)
    print(local_interpretation)

                129  contribution_variable
bio19    289.130000                  0.155
bio4    6977.800000                  0.126
bio6      28.090000                  0.082
bio17    232.820000                  0.081
bio12   1910.610000                  0.077
bio14     64.560000                  0.076
bio3      82.000000                  0.071
bio9      83.540000                  0.068
lon      126.695000                  0.061
bio2      18.800000                  0.060
rh      6386.545833                  0.036
lat       33.462500                  0.023
bio8     239.250000                  0.021
pet       38.402346                  0.020
bio5     254.840000                  0.011
bio11     47.090000                  0.005
alt      338.181818                  0.004
bio7     226.300000                  0.004
bio10    243.840000                  0.001
bio1     140.930000                 -0.000
srad   17364.866667                 -0.001
bio15     43.800000                 -0.015
bio16    79

In [28]:
temp = rf_model.predict_proba(train_test_specs_dict['섬휘파람새'][2])

In [38]:
rf_model.feature_importances_

array([0.0074821 , 0.03411768, 0.04002737, 0.14879243, 0.01609369,
       0.08697278, 0.06995571, 0.01122642, 0.08495232, 0.00707675,
       0.01268287, 0.04364165, 0.03355293, 0.01958289, 0.01721572,
       0.02932447, 0.03529473, 0.03176939, 0.09955532, 0.0381396 ,
       0.05003838, 0.01543903, 0.03007981, 0.02960651, 0.00737943])

In [123]:
instances = train_test_specs_dict['섬휘파람새'][2].iloc[np.array([26,298]),:]
rf = bird_rf['섬휘파람새']

pred_label = rf.predict(instances)
print("Instance 0 prediction:", pred_label[0])
print("Instance 1 prediction:", pred_label[1])
prediction, bias, contributions = ti.predict(rf, instances)

Instance 0 prediction: 1
Instance 1 prediction: 0


In [128]:
for i in range(len(instances)):
    print("Instance", i)
    print("Bias (trainset mean)", bias[i])
    print('Predicted label: ', pred_label[i])
    print("Feature Contribution Observed")
    for c, feature, y in sorted(zip(contributions[i][:,pred_label[i]], list(instances.columns),instances.iloc[i,:]), key=lambda x: -abs(x[0]) ):
        print(feature, '    ', round(c, 2), '    ', y)
    print("-"*20 )

Instance 0
Bias (trainset mean) [0.97478221 0.02521779]
Predicted label:  1
Feature Contribution Observed
bio19      0.15      289.13
bio4      0.13      6977.8
bio6      0.08      28.09
bio17      0.08      232.82
bio12      0.08      1910.61
bio14      0.08      64.56
bio3      0.07      82.0
bio9      0.07      83.54
lon      0.06      126.695
bio2      0.06      18.8
bio13      -0.05      275.39
bio18      -0.05      789.15
rh      0.04      6386.54583333333
bio16      -0.03      791.34
lat      0.02      33.4625
bio8      0.02      239.25
pet      0.02      38.4023456637065
bio15      -0.01      43.8
bio5      0.01      254.84
bio11      0.0      47.09
bio7      0.0      226.3
alt      0.0      338.181818181818
srad      -0.0      17364.866666666698
bio10      0.0      243.84
bio1      -0.0      140.93
--------------------
Instance 1
Bias (trainset mean) [0.97478221 0.02521779]
Predicted label:  0
Feature Contribution Observed
bio4      -0.1      6939.35
bio19      -0.07      187.

In [78]:
prediction

array([[0.1375  , 0.8625  ],
       [0.825625, 0.174375]])

In [80]:
bias + np.sum(contributions, axis=1)

array([[0.1375  , 0.8625  ],
       [0.825625, 0.174375]])

In [82]:
np.sum(contributions, axis=1)

array([[-0.83728221,  0.83728221],
       [-0.14915721,  0.14915721]])

In [83]:
contributions

array([[[ 0.00038901, -0.00038901],
        [-0.06003856,  0.06003856],
        [-0.07087924,  0.07087924],
        [-0.12598187,  0.12598187],
        [-0.01064621,  0.01064621],
        [-0.08204883,  0.08204883],
        [-0.00415856,  0.00415856],
        [-0.0210451 ,  0.0210451 ],
        [-0.06778061,  0.06778061],
        [-0.00087039,  0.00087039],
        [-0.00455097,  0.00455097],
        [-0.07656193,  0.07656193],
        [ 0.04854684, -0.04854684],
        [-0.07557645,  0.07557645],
        [ 0.01498187, -0.01498187],
        [ 0.03322052, -0.03322052],
        [-0.08136622,  0.08136622],
        [ 0.0455013 , -0.0455013 ],
        [-0.15462545,  0.15462545],
        [-0.02032675,  0.02032675],
        [-0.03645668,  0.03645668],
        [ 0.00090102, -0.00090102],
        [-0.06118081,  0.06118081],
        [-0.02287675,  0.02287675],
        [-0.00385139,  0.00385139]],

       [[ 0.02520104, -0.02520104],
        [-0.00819968,  0.00819968],
        [-0.01468639,  0.0

## confidence interval?

In [109]:
#Pseudo code: 
def pred_ci(model, x_val, y_val, percentile = 95):   
    
    """
    x_val = validation input
    percentile = required confidence level
    model = random forest model
    """

    allTree_preds = np.stack([t.predict(x_val) for t in model.estimators_], axis = 0)
#     return allTree_preds
    
    err_down = np.percentile(allTree_preds, (100 - percentile) / 2.0  ,axis=0)
    err_up = np.percentile(allTree_preds, 100- (100 - percentile) / 2.0  ,axis=0)
    
    ci = err_up - err_down
    yhat = model.predict(x_val)
    y = y_val
    
    df = pd.DataFrame()
    df['down'] = err_down 
    df['up'] = err_up
    df['y'] = y
    df['yhat'] = yhat
    df['deviation'] = (df['up'] - df['down'])/df['yhat']
    df.reset_index(inplace=True)
    df_sorted = df.iloc[np.argsort(df['deviation'])[::-1]]
    return df_sorted

In [110]:
a = pred_ci(model=rf, x_val=instances, y_val=np.array([1,0]))

In [111]:
a

,index,down,up,y,yhat,deviation
1,1,0.0,1.0,0,0,inf
0,0,0.0,1.0,1,1,1.0
